In [1]:
import json
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
import torch

/home/chuanbico/miniconda3/envs/vlsp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

# Đọc dữ liệu từ file
train_data = []
with open("duration_training_dataset_normalize.json", "r", encoding="utf-8") as f:  # ← đổi tên file cho đúng với file bạn đang dùng
    for line in f:
        if line.strip():  # Bỏ qua dòng trống
            train_data.append(json.loads(line.strip()))

print(f"Số lượng mẫu: {len(train_data)}")
print("Ví dụ một mẫu đầu tiên:")
print(json.dumps(train_data[0], indent=2, ensure_ascii=False))


Số lượng mẫu: 1490
Ví dụ một mẫu đầu tiên:
{
  "context": "Trong một lớp học, các học sinh đang học về các chủ đề khác nhau. Một số em rất chăm chỉ và thường xuyên hoàn thành bài tập về nhà, trong khi những em khác lại bỏ lỡ nhiều bài. Cô giáo luôn cố gắng khuyến khích cả lớp bằng cách tổ chức các buổi thảo luận thú vị.",
  "labels": [
    "yes",
    "no",
    "yes",
    "no"
  ],
  "options": [
    "168.0 giờ",
    "43800.0 giờ",
    "240.0 giờ",
    "0.003 giờ"
  ],
  "qid": 650,
  "question": "Mất bao lâu để hoàn thành tất cả bài tập về nhà của lớp học?"
}


In [3]:
from collections import Counter

label_counter = Counter()
option_lengths = []

for sample in train_data:
    label_counter.update(sample["labels"])
    option_lengths.append(len(sample["options"]))

print("Tổng số nhãn:")
print(label_counter)
print(f"Số option trung bình mỗi câu hỏi: {sum(option_lengths) / len(option_lengths):.2f}")

Tổng số nhãn:
Counter({'no': 3003, 'yes': 2957})
Số option trung bình mỗi câu hỏi: 4.00


In [4]:
import re

# Từ khóa phân loại context
SHORT_CTX   = ["biểu diễn", "sửa", "trình diễn", "hòa nhạc", "vẽ", "chơi", "làm bài", "bài tập"]
MEDIUM_CTX  = ["tổ chức", "startup", "công ty", "lớp học", "học", "nhóm"]
LONG_CTX    = ["chuẩn bị", "triển lãm", "phát triển", "hoàn thành", "năm", "sự thật", "cuộc sống", "đời sống"]

# Ngưỡng (giờ)
T_SHORT_MED = 168    # 7 ngày
T_MED_LONG  = 720    # 30 ngày

def extract_hours(opt_str):
    m = re.search(r"([\d\.]+)", opt_str)
    return float(m.group(1)) if m else 0.0

def rule_based_predict(sample):
    ctx = sample["context"].lower()
    outs = []
    for opt in sample["options"]:
        h = extract_hours(opt)
        label = "no"
        if h < T_SHORT_MED:
            if any(w in ctx for w in SHORT_CTX):
                label = "yes"
        elif h <= T_MED_LONG:
            if any(w in ctx for w in MEDIUM_CTX):
                label = "yes"
        else:  # h > T_MED_LONG
            if any(w in ctx for w in LONG_CTX):
                label = "yes"
        outs.append(label)
    return outs

# BƯỚC 3: Đánh giá
from sklearn.metrics import classification_report

y_true = []
y_pred = []

for sample in train_data:
    pred = rule_based_predict(sample)
    gold = sample["labels"]
    y_true.extend(gold)
    y_pred.extend(pred)

print(classification_report(y_true, y_pred, target_names=["no", "yes"]))


              precision    recall  f1-score   support

          no       0.51      0.87      0.65      3003
         yes       0.55      0.17      0.26      2957

    accuracy                           0.52      5960
   macro avg       0.53      0.52      0.45      5960
weighted avg       0.53      0.52      0.45      5960



In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)


/home/chuanbico/miniconda3/envs/vlsp/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fr

In [6]:
import json

examples = []
with open("duration_training_dataset_normalize.json", "r", encoding="utf-8") as f:
    for line in f:
        sample = json.loads(line.strip())
        for opt, label in zip(sample["options"], sample["labels"]):
            text = sample["context"] + " " + sample["question"] + " " + opt
            label_id = 1 if label == "yes" else 0
            examples.append({
                "text": text,
                "label": label_id
            })

print(f"Tổng số sample: {len(examples)}")

Tổng số sample: 5960


In [7]:
from datasets import Dataset

dataset = Dataset.from_list(examples)

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)

Map: 100%|██████████| 5960/5960 [00:00<00:00, 7053.31 examples/s]


In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./phobert_duration_hour",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()

/home/chuanbico/miniconda3/envs/vlsp/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 10/5960 [00:03<32:18,  3.07it/s] 

{'loss': 0.7343, 'learning_rate': 4.9916107382550336e-05, 'epoch': 0.02}


  0%|          | 20/5960 [00:06<32:12,  3.07it/s]

{'loss': 0.6583, 'learning_rate': 4.983221476510067e-05, 'epoch': 0.03}


  1%|          | 30/5960 [00:10<31:59,  3.09it/s]

...

                                                   
100%|██████████| 5960/5960 [35:45<00:00,  3.03it/s]

{'eval_loss': 0.6484203934669495, 'eval_runtime': 14.0664, 'eval_samples_per_second': 84.741, 'eval_steps_per_second': 10.593, 'epoch': 10.0}


100%|██████████| 5960/5960 [35:49<00:00,  2.77it/s]

{'train_runtime': 2149.5857, 'train_samples_per_second': 22.181, 'train_steps_per_second': 2.773, 'train_loss': 0.38772331927246695, 'epoch': 10.0}


TrainOutput(global_step=5960, training_loss=0.38772331927246695, metrics={'train_runtime': 2149.5857, 'train_samples_per_second': 22.181, 'train_steps_per_second': 2.773, 'train_loss': 0.38772331927246695, 'epoch': 10.0})

In [9]:
import numpy as np
from sklearn.metrics import classification_report

predictions = trainer.predict(encoded_dataset["test"])
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

print(classification_report(y_true, y_pred, target_names=["no", "yes"]))


100%|██████████| 149/149 [00:14<00:00, 10.55it/s]

              precision    recall  f1-score   support

          no       0.85      0.86      0.86       613
         yes       0.85      0.84      0.85       579

    accuracy                           0.85      1192
   macro avg       0.85      0.85      0.85      1192
weighted avg       0.85      0.85      0.85      1192



In [10]:
trainer.save_model("phobert_duration_hour_model")

# Test with dateset non hour

In [11]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model đã fine-tune
model_path = "./phobert_duration_hour_model"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()  # chuyển sang chế độ dự đoán

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Hàm chuyển nhãn
def id_to_label(pred):
    return "yes" if pred == 1 else "no"

# Hàm dự đoán nhãn cho từng option
def predict_labels(context, question, options):
    labels = []
    for option in options:
        text = context + " " + question + " " + option
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).item()
            labels.append(id_to_label(pred))
    return labels

# Đọc file test
test_data = []
with open("public_test.json", "r", encoding="utf-8") as f:
    for line in f:
        sample = json.loads(line.strip())
        test_data.append(sample)

# Dự đoán và ghi kết quả
with open("non_hour.txt", "w", encoding="utf-8") as f:
    for sample in test_data:
        predicted_labels = predict_labels(sample["context"], sample["question"], sample["options"])
        sample["labels"] = predicted_labels
        f.write(json.dumps(sample, ensure_ascii=False) + "\n")

print("✅ Dự đoán hoàn tất! Kết quả đã lưu vào file: non_hour.txt")


✅ Dự đoán hoàn tất! Kết quả đã lưu vào file: non_hour.txt


In [12]:
import random

# Đọc toàn bộ submission
with open("non_hour.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Chọn ngẫu nhiên 3 dòng
samples = random.sample(lines, k=min(3, len(lines)))

# In nội dung từng dòng
for idx, line in enumerate(samples, 1):
    sample = json.loads(line)
    print(f"\n🧩 Sample {idx} — QID: {sample['qid']}")
    print(f"📘 Context: {sample['context']}")
    print(f"❓ Question: {sample['question']}")
    print("🧠 Options + Labels:")
    for opt, label in zip(sample["options"], sample["labels"]):
        print(f"   - {opt:<15} → {label}")


🧩 Sample 1 — QID: 72
📘 Context: Trong một cuộc họp báo, một phóng viên đã hỏi về các nỗ lực cứu trợ sau một thảm họa thiên nhiên. Các lãnh đạo đã trình bày về công việc của họ, nhấn mạnh tầm quan trọng của việc hỗ trợ cộng đồng bị ảnh hưởng.
❓ Question: Mất bao lâu để hoàn thành các nỗ lực cứu trợ sau thảm họa thiên nhiên?
🧠 Options + Labels:
   - 3 tháng         → no
   - 1 tháng         → no
   - 2 ngày          → yes
   - 5 tháng         → no

🧩 Sample 2 — QID: 66
📘 Context: Trong một buổi biểu diễn nghệ thuật, các nghệ sĩ đã tạo ra những tác phẩm tuyệt đẹp bằng màu sắc và ánh sáng, khiến khán giả không khỏi trầm trồ. Mọi người đều cảm nhận được sự gắn kết và sáng tạo đến từ những tài năng trẻ tuổi.
❓ Question: Mất bao lâu để hoàn thành một buổi biểu diễn nghệ thuật như vậy?
🧠 Options + Labels:
   - 3 giờ           → no
   - 6 giờ           → no
   - 2 tuần          → no
   - 5 tháng         → no

🧩 Sample 3 — QID: 15
📘 Context: Trong một thành phố nhỏ, có một nhóm nghệ sĩ trẻ đang

In [15]:
import json
import re
import numpy as np
from sklearn.metrics import classification_report

def load_labels(path):
    """
    Đọc file JSON (mỗi dòng một object hoặc nối tiếp) và trả về dict: {qid: [0/1,...]}
    với 0 = "no", 1 = "yes".
    """
    text = open(path, 'r', encoding='utf-8').read()
    raw_objs = re.findall(r'\{.*?\}', text, re.DOTALL)
    data = {}
    for obj in raw_objs:
        entry = json.loads(obj)
        # Chuyển "yes"/"no" thành 1/0
        bin_labels = [1 if lab=="yes" else 0 for lab in entry['labels']]
        data[entry['qid']] = bin_labels
    return data

# Đường dẫn tới hai file
gt_path = 'public_test_full_labeled.txt'
sub_path = 'non_hour.txt'

# Load nhãn
gt = load_labels(gt_path)
sub = load_labels(sub_path)

# Xây dựng y_true, y_pred theo thứ tự qid nhất định (vd sort theo qid)
y_true, y_pred = [], []
for qid in sorted(gt.keys()):
    true = gt[qid]
    pred = sub.get(qid, [0]*len(true))
    y_true.extend(true)
    y_pred.extend(pred)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# In classification report
print(classification_report(y_true, y_pred, target_names=["no", "yes"]))


              precision    recall  f1-score   support

          no       0.19      0.71      0.30       206
         yes       0.83      0.32      0.46       906

    accuracy                           0.39      1112
   macro avg       0.51      0.52      0.38      1112
weighted avg       0.71      0.39      0.43      1112



# Dataset Hour

In [16]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model đã fine-tune
model_path = "./phobert_duration_hour_model"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()  # chuyển sang chế độ dự đoán

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Hàm chuyển nhãn
def id_to_label(pred):
    return "yes" if pred == 1 else "no"

# Hàm dự đoán nhãn cho từng option
def predict_labels(context, question, options):
    labels = []
    for option in options:
        text = context + " " + question + " " + option
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).item()
            labels.append(id_to_label(pred))
    return labels

# Đọc file test
test_data = []
with open("public_test_normalize.json", "r", encoding="utf-8") as f:
    for line in f:
        sample = json.loads(line.strip())
        test_data.append(sample)

# Dự đoán và ghi kết quả
with open("hour.txt", "w", encoding="utf-8") as f:
    for sample in test_data:
        predicted_labels = predict_labels(sample["context"], sample["question"], sample["options"])
        sample["labels"] = predicted_labels
        f.write(json.dumps(sample, ensure_ascii=False) + "\n")

print("✅ Dự đoán hoàn tất! Kết quả đã lưu vào file: hour.txt")


✅ Dự đoán hoàn tất! Kết quả đã lưu vào file: hour.txt


In [17]:
import random

# Đọc toàn bộ submission
with open("hour.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Chọn ngẫu nhiên 3 dòng
samples = random.sample(lines, k=min(3, len(lines)))

# In nội dung từng dòng
for idx, line in enumerate(samples, 1):
    sample = json.loads(line)
    print(f"\n🧩 Sample {idx} — QID: {sample['qid']}")
    print(f"📘 Context: {sample['context']}")
    print(f"❓ Question: {sample['question']}")
    print("🧠 Options + Labels:")
    for opt, label in zip(sample["options"], sample["labels"]):
        print(f"   - {opt:<15} → {label}")


🧩 Sample 1 — QID: 131
📘 Context: Một nhóm phóng viên đang chuẩn bị cho một buổi phát sóng trực tiếp về một sự kiện quan trọng sắp diễn ra. Họ cần đảm bảo mọi thứ được sắp xếp hoàn hảo, từ thiết bị đến nội dung phỏng vấn.
❓ Question: Mất bao lâu để chuẩn bị cho một buổi phát sóng trực tiếp?
🧠 Options + Labels:
   - 2.0 giờ         → yes
   - 5.0 giờ         → yes
   - 504.0 giờ       → no
   - 240.0 giờ       → no

🧩 Sample 2 — QID: 163
📘 Context: Một ban nhạc đang chuẩn bị cho buổi trình diễn lớn nhất trong năm. Họ đã lên kế hoạch chi tiết cho từng tiết mục và tập luyện không ngừng nghỉ để mang đến một màn trình diễn xuất sắc.
❓ Question: Mất bao lâu để ban nhạc chuẩn bị cho buổi trình diễn lớn nhất trong năm?
🧠 Options + Labels:
   - 504.0 giờ       → yes
   - 168.0 giờ       → yes
   - 4320.0 giờ      → no
   - 336.0 giờ       → yes

🧩 Sample 3 — QID: 144
📘 Context: Trong một buổi hòa nhạc tại công viên, một nghệ sĩ nổi tiếng đã biểu diễn những bản nhạc quen thuộc khiến khán giả phấ

In [18]:
import json
import re
import numpy as np
from sklearn.metrics import classification_report

def load_labels(path):
    """
    Đọc file JSON (mỗi dòng một object hoặc nối tiếp) và trả về dict: {qid: [0/1,...]}
    với 0 = "no", 1 = "yes".
    """
    text = open(path, 'r', encoding='utf-8').read()
    raw_objs = re.findall(r'\{.*?\}', text, re.DOTALL)
    data = {}
    for obj in raw_objs:
        entry = json.loads(obj)
        # Chuyển "yes"/"no" thành 1/0
        bin_labels = [1 if lab=="yes" else 0 for lab in entry['labels']]
        data[entry['qid']] = bin_labels
    return data

# Đường dẫn tới hai file
gt_path = 'public_test_full_labeled.txt'
sub_path = 'hour.txt'

# Load nhãn
gt = load_labels(gt_path)
sub = load_labels(sub_path)

# Xây dựng y_true, y_pred theo thứ tự qid nhất định (vd sort theo qid)
y_true, y_pred = [], []
for qid in sorted(gt.keys()):
    true = gt[qid]
    pred = sub.get(qid, [0]*len(true))
    y_true.extend(true)
    y_pred.extend(pred)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# In classification report
print(classification_report(y_true, y_pred, target_names=["no", "yes"]))


              precision    recall  f1-score   support

          no       0.31      0.44      0.36       206
         yes       0.86      0.78      0.82       906

    accuracy                           0.72      1112
   macro avg       0.59      0.61      0.59      1112
weighted avg       0.76      0.72      0.73      1112

